In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import lib

### Load Data

Trading on [Xetra](https://www.xetra.com/xetra-en/trading/Trading-calendar-and-trading-hours-22048) is from Mondays to Fridays from 9 until 17:30 CET.

Let's first process data from August first. 

In [ ]:
df = lib.read_date_range(start_date='2019-01-01', end_date='2019-12-31')
df.head()

../data/deutsche-boerse-xetra-pds/2019-04-19/*


In [ ]:
df.shape

Let's see the type of securities on offer. 

In [ ]:
df['SecurityType'].value_counts()

We're only interested in common stock. 

In [ ]:
df = df[df.SecurityType == 'Common stock']

In [ ]:
df.shape

Next, we filter for the trading time, between 08:00 and 20:00. 

In [ ]:
df.set_index('CalcDateTime', drop=True, inplace=True)

In [ ]:
df = df.between_time('08:00', '20:00')

In [ ]:
df.shape

Finally, we remove all auctions, i.e. TradedVolume = 0. 

In [ ]:
df = df[df.TradedVolume > 0]

In [ ]:
df.shape

Let's see the number of unique securities, sorted by volume traded. 

In [ ]:
grouped_securites = df.groupby(['Mnemonic', 'SecurityDesc']).sum()

In [ ]:
grouped_securites.shape

There are 946 unique securities. Let's sort them by volume traded. 

In [25]:
sorted_grouped_securites = grouped_securites.sort_values('TradedVolume', ascending=False)
sorted_grouped_securites.head(10)

,,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
Mnemonic,SecurityDesc,,,,,,
DBK,DEUTSCHE BANK AG NA O.N.,5.963460e+05,5.967346e+05,5.959464e+05,5.963390e+05,3.009862e+09,2160011.0
DTE,DT.TELEKOM AG NA,1.096916e+06,1.097261e+06,1.096567e+06,1.096914e+06,2.133462e+09,1144762.0
SNH,"STEINHOFF INT.HLDG.EO-,50",1.339673e+03,1.341808e+03,1.337258e+03,1.339460e+03,2.112372e+09,72991.0
CBK,COMMERZBANK AG,3.397013e+05,3.399209e+05,3.394780e+05,3.397011e+05,1.999202e+09,1149542.0
EOAN,E.ON SE NA O.N.,7.222737e+05,7.225034e+05,7.220414e+05,7.222732e+05,1.591527e+09,917812.0
LHA,LUFTHANSA AG VNA O.N.,7.910846e+05,7.916316e+05,7.905196e+05,7.910812e+05,1.427914e+09,1281523.0
IFX,INFINEON TECH.AG NA O.N.,1.521153e+06,1.521956e+06,1.520339e+06,1.521151e+06,1.266239e+09,1651415.0
DAI,DAIMLER AG NA O.N.,2.927073e+06,2.928782e+06,2.925333e+06,2.927042e+06,9.103711e+08,2460598.0
AT1,"AROUNDTOWN EO-,01",3.982076e+05,3.983602e+05,3.980568e+05,3.982080e+05,8.453975e+08,530283.0


The current analysis will be limited to the Top 100 stocks. 

In [26]:
securities = list(sorted_grouped_securites.index.get_level_values('Mnemonic')[:100])

In [ ]:
import pickle
with open("securities.txt", "wb") as f:   #Pickling
    pickle.dump(securities, f)

Now we limit our dataset to this 100 securities. 

In [ ]:
df = df[df['Mnemonic'].isin(securities)]

In [ ]:
df.shape

In [ ]:
df.head()

Sweet! Exporting this to parquet. 

In [30]:
df.to_parquet('../data/processed_data/20200904/top100stocks_cleaned_2019.parquet')